In [ ]:
import numpy as np
import pandas as pd
from mnist import load_mnist

from daml.metrics.drift import CVMDrift, KSDrift, MMDDrift

In [ ]:
# Create
rng = np.random.default_rng(33)
size = 1000

test_images, labels = load_mnist(size)
test_images = test_images / 255

rng.shuffle(test_images)
print(test_images.shape)

In [ ]:
ks = KSDrift(test_images)
cvm = CVMDrift(test_images)
mmd = MMDDrift(test_images)

In [ ]:
ksr = ks.predict(test_images * 1.01)
cvmr = cvm.predict(test_images * 1.01)
mmdr = mmd.predict(test_images * 1.01)

any_drift = mmdr["is_drift"] or cvmr["is_drift"] or ksr["is_drift"]
print(any_drift)

drift_preds = {"Maximum Mean Discrepency": mmdr, "Cramér-von Mises": cvmr, "Kolmogorov-Smirnov": ksr}

In [ ]:
from gradient.slide_deck.shapes import SubText, Table, Text, TextContent
from gradient.slide_deck.slidedeck import DefaultGradientSlideLayouts


def generate_drift_report_table(drift_preds: dict) -> pd.DataFrame:
    drift_table = pd.DataFrame(
        {
            "Method": list(drift_preds),
            "Has drifted?": ["Yes" if preds["is_drift"] else "No" for preds in drift_preds.values()],
            "Test statistic": [np.mean(preds["distance"]) for preds in drift_preds.values()],
            "Threshold": [preds["threshold"] for preds in drift_preds.values()],
            "Significance": [np.mean(preds["p_val"]) for preds in drift_preds.values()],
        }
    )
    return drift_table


def generate_drift_report_slide_kwargs(drift_preds: dict) -> dict:
    content = [
        "Operational dataset ",
        SubText(f"{'has' if any_drift else 'has not'}", bold=True),
        " drifted from Development dataset",
    ]

    kwargs = {
        "title": "Drift Detection: Summary",
        "layout": DefaultGradientSlideLayouts.CONTENT_DEFAULT,
        "placeholder_fillings": [TextContent(lines=[Text(content=content)])],
        "additional_shapes": [
            Table(
                dataframe=generate_drift_report_table(drift_preds).round(4),
                fontsize=16,
                left=2.0,
                top=2.0,
                width=9.0,
                height=4.0,
            ),
        ],
    }
    return kwargs

In [ ]:
from pathlib import Path

from gradient.slide_deck.slidedeck import DEFAULT_GRADIENT_PRESENTATION_TEMPLATE_PATH, SlideDeck

example_directory = Path.cwd() / "report_drift_example"
example_directory.mkdir(parents=True, exist_ok=True)

deck = SlideDeck(presentation_template_path=DEFAULT_GRADIENT_PRESENTATION_TEMPLATE_PATH)

deck.add_slide(**generate_drift_report_slide_kwargs(drift_preds))

deck.save(
    output_directory=example_directory,
    name="report_drift_example",
)